In [1]:
import pandas as pd
import numpy as np

import pyspark as ps
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

import nltk

seed = 2333
np.random.seed(seed)
data_path = '/home/raku/kaggleData/quora/'


In [30]:
#sample = pd.read_csv(data_path+"sample.csv")
stemmed_sample = pd.read_pickle(data_path+'sample_stemmed_questions.pkl')
#processed_sample = sample.merge(stemmed_sample)
sample = stemmed_sample
print sample.columns

Index([u'id', u'question_1_stemmed', u'question_2_stemmed'], dtype='object')


In [17]:
#function for sharing
def word_match_share(text_a,text_b):
    a = set(text_a.split())
    b = set(text_b.split())
    if len(a) == 0 or len(b) == 0:return 0
    return len(a.intersection(b)) *1.0 / (len(a)+len(b))

word_match_share_udf = udf(word_match_share)

#function for filtering stop-words
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

def filter_stop_words(text):
    return " ".join([i for i in text.split() if i not in stops])

filter_stop_words_udf = udf(filter_stop_words)

len_udf = udf(len)

def word_length(text):
    word_list = text.split()
    return len(word_list)

word_length_udf = udf(word_length)

In [ ]:
#conf = ps.SparkConf().setAppName('jupyter_backend').setMaster('local[7]')
#sc = ps.SparkContext(conf=conf)
#sqlContext = ps.sql.SQLContext(sc)

In [31]:
sample_sp = sqlContext.createDataFrame(sample)
sample_sp=sample_sp.withColumn('raw_sharing',word_match_share_udf(sample_sp[u'question_1_stemmed'],sample_sp[u'question_2_stemmed']))
#filter stop words and get the filtered_sharing
sample_sp=sample_sp.withColumn('filtered_q1',filter_stop_words_udf(sample_sp['question_1_stemmed']))
sample_sp=sample_sp.withColumn('filtered_q2',filter_stop_words_udf(sample_sp['question_2_stemmed']))
sample_sp=sample_sp.withColumn('filtered_sharing',word_match_share_udf(sample_sp[u'filtered_q1'],sample_sp[u'filtered_q2']))

In [49]:
sample_sp=sample_sp.withColumn('word_length_q1',word_length_udf(sample_sp['question_1_stemmed']))
sample_sp=sample_sp.withColumn('word_length_q2',word_length_udf(sample_sp['question_2_stemmed']))

In [46]:
sample_sp=sample_sp.withColumn('char_length_q1',len_udf(sample_sp['question_1_stemmed']))
sample_sp=sample_sp.withColumn('char_length_q2',len_udf(sample_sp['question_2_stemmed']))

In [50]:
sample_sp=sample_sp.withColumn('word_len_diff',sample_sp['word_length_q1']-sample_sp['word_length_q2'])
sample_sp=sample_sp.withColumn('word_len_rate',sample_sp['word_length_q1']*1.0/sample_sp['word_length_q2'])
sample_sp=sample_sp.withColumn('word_len_diff_normal',
                               (sample_sp['word_length_q1']-sample_sp['word_length_q2'])*1.0/(sample_sp['word_length_q1']+sample_sp['word_length_q2']))
sample_sp=sample_sp.withColumn('word_len_diff_normal',ps.sql.functions.abs(sample_sp['word_len_diff_normal']))
sample_sp=sample_sp.withColumn('word_len_diff',ps.sql.functions.abs(sample_sp['word_len_diff']))
sample_sp=sample_sp.withColumn('word_len_sum',sample_sp['word_length_q1']+sample_sp['word_length_q2'])

In [67]:
sample_sp=sample_sp.withColumn('char_len_diff',sample_sp['char_length_q1']-sample_sp['char_length_q2'])
sample_sp=sample_sp.withColumn('char_len_rate',sample_sp['char_length_q1']*1.0/sample_sp['char_length_q2'])
sample_sp=sample_sp.withColumn('char_len_diff_normal',
                               (sample_sp['char_length_q1']-sample_sp['char_length_q2'])*1.0/(sample_sp['char_length_q1']+sample_sp['char_length_q2']))
sample_sp=sample_sp.withColumn('char_len_diff_normal',ps.sql.functions.abs(sample_sp['char_len_diff_normal']))
sample_sp=sample_sp.withColumn('char_len_diff',ps.sql.functions.abs(sample_sp['char_len_diff']))
sample_sp=sample_sp.withColumn('char_len_sum',sample_sp['char_length_q1']+sample_sp['char_length_q2'])

In [71]:
filtered_fields=['filtered_q1','filtered_q2']
sharing = ['raw_sharing','filtered_sharing']
word_lengths = ['word_length_q1','word_length_q2','word_len_diff','word_len_rate','word_len_diff_normal','word_len_sum']
char_lengths = ['char_length_q1','char_length_q2','char_len_diff','char_len_rate','char_len_diff_normal','char_len_sum']

In [72]:
sample_res = sample_sp[filtered_fields+sharing+word_lengths+char_lengths].toPandas()

In [75]:
sample_res[filtered_fields].to_pickle(data_path+'sample_filtered.pkl')
sample_res[sharing].to_pickle(data_path+'sample_sharing.pkl')
sample_res[word_lengths].to_pickle(data_path+'sample_word_length.pkl')
sample_res[char_lengths].to_pickle(data_path+'sample_char_lengths.pkl')


In [ ]:
print 1 !=